***<h1 style="text-align: center;">LangChain Basics</h1><hr>***

## Setup Open AI API key

In [1]:
import sys

sys.path.insert(0, 'D:/personal-git')
from key_api import apiKey

import os
os.environ['OPENAI_API_KEY'] = apiKey

## Install necessary modules

In [3]:
!python -m pip install langchain
!python -m pip install OpenAI